# Scraper for ICASSP from IEEE

> "From a pre-extracted page, content seems to be fetched separately"

- hidden: true
- toc: false
- badges: true
- categories: [scraper, icassp]

In [2]:
from bs4 import BeautifulSoup

The temporary file below is extracted from [IEEE](https://ieeexplore.ieee.org/xpl/conhome/1000002/all-proceedings)

In [20]:
with open("/tmp/icassp") as inf:
    html = inf.read()

In [21]:
soup = BeautifulSoup(html, 'html.parser')

In [22]:
top = soup.find("div", {"class": "issue-list-container"})

In [23]:
import re

In [35]:
data = []
data.append("| Year | Volume | DOI | Issue |")
data.append("|------|--------|-----|-------|")
for li in top.find_all("li", {"_ngcontent-ftl-c305": ""}):
    raw = li.text.strip()
    m1 = re.search("^ICASSP (20\d\d).*", raw)
    if m1:
        year = m1.group(1)
    m2 = re.search("^((?:20|19)\d\d) IEEE.*", raw)
    if m2:
        year = m2.group(1)
    m3 = re.search("^ICASSP '(\d\d).*", raw)
    if m3:
        year = m3.group(1)
        print(year)
    #print(raw)


87
86
85
84
83
82
81
80
79
78
77
76


In [18]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown("\n".join(data)))